# Corretor ortográfico
A ideia desse notebook é implementar corretor ortográfico automático pro texto das leis, a fim de analisar se isto impacta na qualidade da representação dos documentos

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Importa arquivos
# Descomente pra usar no Google Colab
# from google.colab import files
# import os.path

arq_leis = 'leis.json'

# if (not os.path.isfile(arq_leis)):
#     uploaded = files.upload()

In [4]:
leis = pd.read_json(arq_leis)
leis.drop(['documento'], inplace=True, axis=1)
print(leis.info())
print(leis.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6033 entries, 0 to 6032
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   titulo     6033 non-null   object
 1   categoria  6033 non-null   object
 2   resumo     6033 non-null   object
 3   texto      6033 non-null   object
dtypes: object(4)
memory usage: 188.7+ KB
None
titulo       6033
categoria       8
resumo       4961
texto        6029
dtype: int64


In [5]:
leis

,titulo,categoria,resumo,texto
0,"DECRETO Nº 8854, de 28 de fevereiro de 2013.",Decretos,DELEGA COMPETÊNCIA À SECRETARIA MUNICIPAL DE P...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
1,"DECRETO Nº 8849, de 25 de fevereiro de 2013.",Decretos,ABRE CRÉDITO SUPLEMENTAR AO ORÇAMENTO DO MUNIC...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
2,"DECRETO Nº 8853, de 27 de fevereiro de 2013.",Decretos,NOMEIA MEMBROS DO CONSELHO MUNICIPAL DE DESENV...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
3,"DECRETO Nº 8967, de 17 de julho de 2013",Decretos,ALTERA O QUADRO DE DETALHAMENTO DE DESPESA DO ...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
4,"DECRETO Nº 8982, de 30 de julho de 2013",Decretos,AUTORIZA O FUNCIONAMENTO DE ESTABELECIMENTOS C...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
...,...,...,...,...
6028,RESOLUÇÃO Nº 125/1980,Resoluções,DISPÕE SOBRE A CONCESSÃO DE TÍTULO DE CIDADÃO ...,Faço saber que a Câmara Municipal aprovou e eu...
6029,RESOLUÇÃO Nº 403/2003,Resoluções,AUTORIZA A MESA DIRETIVA DO PODER LEGISLATIVO ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6030,RESOLUÇÃO Nº 492/2014,Resoluções,INSTITUI A SEGUNDA SEMANA DO MÊS DE AGOSTO EM ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6031,RESOLUÇÃO Nº 382/2001,Resoluções,CRIA A MEDALHA VEREADOR DIVAL FIGUEIREDO MACHA...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."


In [6]:
# Exemplo de texto de lei
leis.loc[len(leis)-1, 'texto']

'A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, Inciso V, da Lei Municipal nº37, de 05 de Abril de 1990 e, artigos 287, § 2º e, 420, do Regimento Interno, promulga a seguinte Resolução:\n\nArt. 1ºDê-se aos dispositivos abaixo mencionados, da Resolução nº393/2002 - Regimento Interno, as seguintes redações:\n\n"Art. 7º A Mesa Diretora da Câmara compor-se-á do Presidente, Primeiro e Segundo Secretários, com mandato de 02 ( dois ) anos, admitida a recondução para a eleição subsequente.\n\n§ 4º Se, hora regimental, não estiver presente o Presidente, abrirá os trabalhos o Vice-Presidente ou, na falta deste, o Primeiro ou Segundo Secretários, na sequência, ou ainda, caso estes não estejam presentes, o Vereador mais votado nas eleições municipais."\n\n"Art. 33 Compete, privativamente, ao Vice-Presidente:"\n\n"Art. 36 ...\n\nI - ...\n\ne) acompanhar e supervisionar a Ata da Sessão, proceder a sua leitura e assiná-la depois do Presidente e do Vice-Presiden

In [29]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
# import unicodedata

def limpa_texto(text, return_string=True) -> str or list:
    if (type(text) == float):
        return ''

    # Remove pontuacao, digitos e espacos
    text = re.sub(r'[0-9]+', ' NUM ', text.lower())
    text = ' '.join(re.findall(r'\b[A-Za-zÀ-ú]+[-A-Za-zÀ-ú]*', text))
    
    # Remove acentos, cedilhas etc
    # nfkd_form = unicodedata.normalize('NFKD', text)
    # text = ''.join([char for char in nfkd_form if not unicodedata.combining(char)])

    # Remove stopwords
    my_stopwords = stopwords.words('portuguese')
    my_words = ['desta', 'destas', 'deste', 'destes', 'qualquer', 'das', 'seguintes', 
    'i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x',
    'xi', 'xii', 'xiii', 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx',
    'b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 
    't', 'u', 'w', 'x', 'y', 'z']
    my_stopwords = my_stopwords + my_words

    text = [word for word in text.split() if word not in my_stopwords]

    if return_string:
        return ' '.join(text)
    else:
        return text

[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Semelhança de documentos

In [30]:
leis['texto_limpo'] = leis['texto'].apply(limpa_texto)

In [31]:
# Constrói matriz de documentos
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
representacao_contagem = vectorizer.fit_transform(leis['texto_limpo'])
representacao_contagem

<6033x30210 sparse matrix of type '<class 'numpy.int64'>'
	with 625461 stored elements in Compressed Sparse Row format>

In [32]:
vocabulario = vectorizer.vocabulary_
print(len(vocabulario))

30210


In [33]:
dicionario = []
with open('.dicio/palavras.txt') as f:
    dicionario = [i for i in f.read().splitlines()]
print(len(dicionario))

320139


In [34]:
dicionario = set(map(limpa_texto, dicionario))
len(dicionario)

318280

In [35]:
# comeca_o = [i for i in dicionario if (i[0:2]== 'us')]
# comeca_o

In [36]:
palavras_erradas = [palavra for palavra in vocabulario if palavra not in dicionario]
print(len(palavras_erradas))
palavras_erradas

19430


['uso',
 'atribuições',
 'considerando',
 'denominado',
 'oferta',
 'aludido',
 'num',
 'dispõe',
 'antidrogas',
 'estabelece',
 'outra',
 'desenvolvimento',
 'políticas',
 'drogas',
 'ilícitas',
 'decreta',
 'comad',
 'respectivas',
 'entrará',
 'revogadas',
 'disposições',
 'lucena',
 'legais',
 'contida',
 'detalhamento',
 'class',
 'programática',
 'econômica',
 'formatada',
 'técnica',
 'leismunicipais',
 'disponíveis',
 'despesas',
 'decorrentes',
 'presente',
 'correrão',
 'anulações',
 'dotações',
 'detalhadas',
 'contabilidade',
 'autorizada',
 'contábeis',
 'necessários',
 'cumprimento',
 'estratégica',
 'fomento',
 'política',
 'indicações',
 'formuladas',
 'entes',
 'governamentais',
 'integram',
 'nomeadas',
 'integrarem',
 'representando',
 'settdec',
 'márcia',
 'cristina',
 'campodônio',
 'eloy',
 'planejamento',
 'brito',
 'fraga',
 'maia',
 'luis',
 'venas',
 'universidade',
 'uefs',
 'celismara',
 'morais',
 'subaé',
 'cedraz',
 'vaccarezza',
 'baiana',
 'ebda',
 'ca

In [37]:
dic_ugc = []
with open('.dicio/dicio_UGC.txt') as f:
    dic_ugc = [linha.split(',')[0] for linha in f]
len(dic_ugc)

549053

In [38]:
palavras_erradas_ugc = [palavra for palavra in palavras_erradas if palavra not in dic_ugc]
print(len(palavras_erradas_ugc))
palavras_erradas_ugc

5259


['comad',
 'class',
 'leismunicipais',
 'settdec',
 'campodônio',
 'venas',
 'uefs',
 'celismara',
 'subaé',
 'cedraz',
 'vaccarezza',
 'ebda',
 'cifs',
 'acefs',
 'sincont',
 'sicomfs',
 'setcarfs',
 'délcio',
 'adalton',
 'sindihr',
 'sindvest',
 'ane',
 'meirevam',
 'claudiana',
 'edmeia',
 'ildes',
 'senaes',
 'comprodec',
 'obpi',
 'arcênio',
 'suzart',
 'osmário',
 'aidil',
 'giselle',
 'pregoeira',
 'merces',
 'elidiane',
 'sirleide',
 'dalei',
 'orgânicado',
 'capítuloi',
 'pmi',
 'ppps',
 'cgpm',
 'ueppp',
 'gts',
 'capítuloii',
 'capítuloiii',
 'combiná',
 'encaminhá',
 'los',
 'capítuloiv',
 'evantamentos',
 'capítulov',
 'tama',
 'lt',
 'issqn',
 'subempreitada',
 'spe',
 'reinvidicações',
 'dival',
 'mensag',
 'izaltina',
 'claudemiro',
 'pelúcio',
 'elisiana',
 'lesgisl',
 'anisía',
 'datilog',
 'financ',
 'uilma',
 'doranei',
 'zenilda',
 'eunira',
 'deb',
 'ofic',
 'egisl',
 'ol',
 'erideth',
 'tesour',
 'motorist',
 'smt',
 'apresentá',
 'identificá',
 'ebenezer',
 'tu

In [39]:
# Soluções prontas: spellchecker, conferir no Google "Did you mean:"
from spellchecker import SpellChecker
spellc = SpellChecker(language='pt')
erradas = spellc.unknown(['dalei', 'orgânicado', 'capítuloi', 'capítuloiii', 'combiná', 'encaminhá', 'los', 
                            'reinvidicações', 'mensag', 'bonito', 'orgânicamunicipal', 'segundafeira', 'jayana'])
# Não pode tirar hifem no meio de palavras. Consertar no método limpa texto
for palavra in erradas:
    print(f'Palavra da vez: {palavra}')
    print(spellc.correction(palavra))
    print(spellc.candidates(palavra))

Palavra da vez: reinvidicações
reivindicações
{'reivindicações'}
Palavra da vez: segundafeira
segundafeira
{'segundafeira'}
Palavra da vez: capítuloi
capítulo
{'capítulo', 'capítulos'}
Palavra da vez: orgânicamunicipal
orgânicamunicipal
{'orgânicamunicipal'}
Palavra da vez: jayana
sacana
{'bacana', 'abana', 'joana', 'aayan', 'jaya', 'mayan', 'sacana', 'mayans', 'rayna', 'savana', 'mañana', 'alana', 'jaffna', 'catana', 'havana', 'juana', 'katana', 'banana', 'cabana', 'jayne', 'jana'}
Palavra da vez: encaminhá
encaminhar
{'encaminhar'}
Palavra da vez: mensag
mensal
{'mensal', 'mensa'}
Palavra da vez: capítuloiii
capítuloiii
{'capítuloiii'}
Palavra da vez: orgânicado
organizado
{'orgânicas', 'organizado', 'orgânica', 'orgânico'}
Palavra da vez: dalei
falei
{'falei', 'dalek', 'dale', 'daley', 'dalai', 'dali', 'darei'}
Palavra da vez: combiná
combina
{'combina', 'combine'}


## Construindo o corretor

Os passos são:
- gerar palavras que estão próximas (a 1 ou 2 edits de distância)
    - separando a palavra
    - removendo um caracter
    - inserindo um caracter
    - substituindo um caracter por outro (removendo + inserindo)
- atribuir probabilidade entre as palavras candidatas com base no corpus
- substituir pela palavra mais provável

Alguns problemas: 
- não queremos modificar nomes próprios, nem de pessoas nem de locais.
- Da mesma forma, não queremos modificar siglas: uefs, smtt

Fontes: 
[NLP por Deeplearnin.ai, no Coursera](https://www.coursera.org/learn/probabilistic-models-in-nlp), 
[Corretor de Peter Norvig](http://norvig.com/spell-correct.html)


In [ ]:
# Gerando palavras próximas
def gera_palavras_proximas(palavra):
    inserts = 'abcdefghijklmnopqrstuvwxyz-'
    separacoes = [(palavra[:i], palavra[i:]) for i in range(1,len(palavra))]
    remocoes = [esquerda + direita[1:] for esquerda, direita in separacoes if direita]
    insercoes = [palavra[:i] + caracter + palavra[i:] for i in range(len(palavra))]
    return [separacoes + remocoes + insercoes]

def gera_palavras_dois_edits(palavra):
    return [dois_edits for um_edit in gera_palavras_proximas(palavra) for dois_edits in gera_palavras_proximas(um_edit)]

# Atribuir probabilidade para palavras geradas
# comparar com vocabulário do corpus
qtd_palavras = representacao_contagem.sum()
contagem_palavra = sum(representacao_contagem)
def calcula_probabilidades(palavra):
    probabilidades_candidatas = {}
    for candidata in gera_palavras_dois_edits(palavra):
        if candidata in vocabulario:
            probabilidades_candidatas[candidata] = contagem_palavra[vocabulario[candidata]]
    return probabilidades_candidatas